In [ ]:
pip install chess tqdm

In [ ]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.optim.lr_scheduler import OneCycleLR # type: ignore
import math
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
import tqdm # type: ignore
from dataset import ChessDataset
from model import ChessModel
from helper_funcs import collect_unique_moves, create_input_for_nn
from helper_funcs import process_data_and_save_chunks
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = '/content/drive/MyDrive/models'
os.makedirs(save_dir, exist_ok=True) 

In [ ]:
# Check for GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using cuda backend ")
else:
    device = torch.device('cpu')
    print("MPS backend not available. Using CPU.")

num_classes = 1941
    
# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()

data_chunk_files = [f'data_chunk_{i}.npz' for i in range(21)]

In [ ]:
num_epochs = 2

batch_size = 320  # Ensure this matches your DataLoader batch_size
total_batches_per_epoch = 0
for data_chunk_file in data_chunk_files:
    data = np.load(data_chunk_file)
    num_samples = data['X'].shape[0]
    num_batches = math.ceil(num_samples / batch_size)
    total_batches_per_epoch += num_batches

total_steps = num_epochs * total_batches_per_epoch

optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = OneCycleLR(optimizer, max_lr=0.001, total_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    total_batches = 0
    for data_chunk_file in tqdm.tqdm(data_chunk_files, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # Load data chunk
        data = np.load(data_chunk_file)
        X = torch.tensor(data['X'], dtype=torch.float32).to(device)
        y = torch.tensor(data['y'], dtype=torch.long).to(device)
        # Create Dataset and DataLoader
        dataset = ChessDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=320, num_workers=0, shuffle=True)
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            optimizer.zero_grad()
            outputs = model(inputs)  # Raw logits
            # Compute loss
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()  # Update learning rate
            running_loss += loss.item()
            total_batches += 1
        # Free up memory
        del X, y, dataset, dataloader
        torch.cuda.empty_cache()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)
    avg_loss = running_loss / total_batches
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Time: {minutes}m{seconds}s')

In [ ]:
save_path = os.path.join(save_dir, 'mark4.3-25e-500k.pth')
torch.save(model.state_dict(), save_path)
print(f'Model saved to {save_path}')